In [38]:
import labhelper as hp
from labhelper.unit_systems.SI import *
from fractions import Fraction
from math import sqrt

In [39]:
def no_simplify(numerator: Quantity, denominator: Quantity) -> Quantity:
    print(numerator.expected_units)
    print(denominator.expected_units)

In [40]:
def prime_factorization_int(n: int) -> list[tuple[int, int]]:
    if n == 1:
        return []
    elif n == 0:
        return []

    fac = []
    powers = []
    count = 0
    while n % 2 == 0:
        count += 1
        n = n // 2
    fac.append(2) if count > 0 else 0
    powers.append(count) if count > 0 else 0
    # n must be odd at this point
    # so a skip of 2 ( i = i + 2) can be used
    for i in range(3,int(sqrt(n))+1,2):
        # while i divides n , add i to the list
        count = 0
        while n % i== 0:
            count += 1
            n = n // i
        fac.append(i) if count > 0 else 0
        powers.append(count) if count > 0 else 0
    # Condition if n is a prime
    # number greater than 2
    if n > 2:
        fac.append(n)
        powers.append(1) 
    return [(factor, power) for factor, power in zip(fac, powers)]


def prime_factorization(n: Fraction) -> list[tuple[int, int]]:
    positive_factors = []
    negative_factors = []

    positive_factors = prime_factorization_int(n.numerator)
    negative_factors = [(factor, -power) for factor, power in prime_factorization_int(n.denominator)]
    final = positive_factors + negative_factors
    final.sort()
    return final

def custom_factors(n: Fraction, custom_factors: list[Fraction]): # TODO: Check this because it's probably wildly unoptimized
    num_factors = prime_factorization(n)
    factors_check = [prime_factorization(custom) for custom in custom_factors]
    final = []
    for fac, custom in zip(factors_check, custom_factors):
        #if all(prime_fac[0] in [e[0] for e in num_factors] for prime_fac in fac):
        not_matching = [a for a in fac if a[0] not in [b[0] for b in num_factors] ]
        if not_matching:
            final.append((custom, 1))
            common_num = [a for a in num_factors if a[0] in [b[0] for b in fac]]
            different_num = [a for a in num_factors if a not in common_num]
            common_fac = [a for a in fac if a[0] in [b[0] for b in num_factors]]
            different_fac = [a for a in fac if a not in common_fac]
            common = [(a[0], a[1] - b[1]) for a, b in zip(common_num, common_fac)]
            num_factors = common + different_num + [(a[0], -a[1]) for a in different_fac]
            continue
            
        matching = [a for a in num_factors if a[0] in [b[0] for b in fac]]
        not_matching = [a for a in num_factors if a not in matching]
        power = min([prime_n[1]//prime_fac[1] for prime_fac, prime_n in zip(fac, matching)], key=abs)
        if power == 0:
            num_factors = [(prime_n[0], prime_n[1] - prime_fac[1]) for prime_fac, prime_n in zip(fac, matching) if prime_n[1] - prime_fac[1] != 0] + not_matching
            final.append((custom, 1))
        else:
            num_factors = [(prime_n[0], prime_n[1] - prime_fac[1]*power) for prime_fac, prime_n in zip(fac, matching) if prime_n[1] - prime_fac[1]*power != 0] + not_matching
            final.append((custom, power))
    return final + num_factors


In [56]:
def prime_factorization_int_test(n: int) -> tuple[list[int], list[int]]:
    if n == 1:
        return [], []
    elif n == 0:
        return [], []

    fac = []
    powers = []
    count = 0
    while n % 2 == 0:
        count += 1
        n = n // 2
    fac.append(2) if count > 0 else 0
    powers.append(count) if count > 0 else 0
    # n must be odd at this point
    # so a skip of 2 ( i = i + 2) can be used
    for i in range(3,int(sqrt(n))+1,2):
        # while i divides n , add i to the list
        count = 0
        while n % i== 0:
            count += 1
            n = n // i
        fac.append(i) if count > 0 else 0
        powers.append(count) if count > 0 else 0
    # Condition if n is a prime
    # number greater than 2
    if n > 2:
        fac.append(n)
        powers.append(1) 
    return fac, powers

def prime_factorization_test(n: Fraction) -> tuple[list[int], list[int]]:
    positive_factors, positive_powers = prime_factorization_int_test(n.numerator)
    negative_factors, negative_powers = prime_factorization_int_test(n.denominator)
    return positive_factors + negative_factors, positive_powers + [-e for e in negative_powers]

def test(n: Fraction, custom_factors: list[Fraction]):
    num_factors, num_powers = prime_factorization_test(n)
    factors_of_factors = [prime_factorization_test(factor) for factor in custom_factors]
    factors = []
    for factor_factors, factor_powers, custom_factor in zip([a[0] for a in factors_of_factors], [a[1] for a in factors_of_factors], custom_factors):
        common_factors = set(num_factors).intersection(factor_factors)
        common_ids_num = [num_factors.index(x) for x in common_factors]
        different_ids_num = [x for x in range(len(num_factors)) if x not in common_ids_num]
        common_ids_fac = [factor_factors.index(x) for x in common_factors]
        different_ids_fac = [x for x in range(len(factor_factors)) if x not in common_ids_fac]
        #different_factors = [num_factors[i] for i in different_ids_num] + [factor_factors[i] for i in different_ids_fac]
        if different_ids_fac: # We divide by the factor since the numbers are not compatible
            num_powers = [num_powers[i] - factor_powers[j] for i, j in zip(common_ids_num, common_ids_fac) if num_powers[i] - factor_powers[j] != 0] + [num_powers[i] for i in different_ids_num]
            num_factors = list(common_factors) + [num_factors[i] for i in different_ids_num]
            factors += list(zip([factor_factors[i] for i in different_ids_fac], [-factor_powers[i] for i in different_ids_fac]))
            factors.append((custom_factor, 1))
            continue

        power = min([num_powers[i]//factor_powers[j] for i, j in zip(common_ids_num, common_ids_fac)], key=abs)
        
    factors += list(zip(num_factors, num_powers))
    factors.sort()
    return factors

print(custom_factors(Fraction(20), [Fraction(20**2)]))
print(test(Fraction(20), [Fraction(20**2)]))

[(Fraction(400, 1), 1), (2, -2), (5, -1)]
0
[(2, 2), (5, 1)]
